In [4]:
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox as messagebox
from tkinter import filedialog
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import collections
import pickle
from scapy.all import *
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
def build_dict(file,recordcount=500):
    record1 = {}
    for feat in ['eth_dst','eth_src','eth_type','ip_version','ip_ihl','ip_tos','ip_len','ip_id','ip_flags','arp_src_ipv4','arp_dst_ipv4',
                 'arp_src_mac','arp_dst_mac','ip_frag','ip_ttl','ip_proto','ip_chksum','ip_src','ip_dst','udp_sport','udp_dport',
                 'udp_len','udp_chksum','icmp','tcp','tcp_sport','tcp_flags_syn','tcp_dport','tcp_seq','tcp_ack','tcp_dataofs','tcp_reserved',
                 'tcp_flags','tcp_window','tcp_chksum','tcp_urgptr','framenumber','arp','tcp_flag_reset']:
        record1[feat] = -1
    
    pcap = rdpcap(file)
   
    df = pd.DataFrame()
    CNT = [0]
    coun=0
    for pkt in pcap:
        if(coun<recordcount):
            record = record1
            coun=coun+1
            
            if pkt.haslayer('frame_info'):
                # Extract the frame number from the frame_info attribute
                record['framenumber'] = pkt.frame_info.number
                
            if pkt.haslayer(Ether):
                record['eth_dst'] = pkt[Ether].dst
                record['eth_src'] = pkt[Ether].src
                record['eth_type'] = pkt[Ether].type
                
            if pkt.haslayer(IP):
                record['ip_version'] = pkt[IP].version
                record['ip_ihl'] = pkt[IP].ihl
                record['ip_tos'] = pkt[IP].tos
                record['ip_len'] = pkt[IP].len
                record['ip_id'] = pkt[IP].id
                record['ip_flags'] = pkt[IP].flags
                record['ip_frag'] = pkt[IP].frag
                record['ip_ttl'] = pkt[IP].ttl
                record['ip_proto'] = pkt[IP].proto
                record['ip_chksum'] = pkt[IP].chksum
                record['ip_src'] = pkt[IP].src
                record['ip_dst'] = pkt[IP].dst
                
                
            if pkt.haslayer(UDP):
                record['udp_sport'] = pkt[UDP].sport 
                record['udp_dport'] = pkt[UDP].dport
                record['udp_len'] = pkt[UDP].len
                record['udp_chksum'] = pkt[UDP].chksum
            
                
            if pkt.haslayer(ICMP):
                record['icmp'] = 1
            
            if pkt.haslayer(ARP):
                arp_layer = pkt[ARP]
                record['arp'] = 1
                record['arp_src_ipv4'] = arp_layer.psrc
                record['arp_dst_ipv4'] = arp_layer.pdst
                record['arp_src_mac'] = arp_layer.hwsrc
                record['arp_dst_mac'] = arp_layer.hwdst
                
                
            if pkt.haslayer(TCP):
                if pkt[TCP].haslayer(Raw):
                    record['tcp']=1
                record['tcp_sport'] = pkt[TCP].sport
                record['tcp_dport'] = pkt[TCP].dport
                record['tcp_seq'] = pkt[TCP].seq
                record['tcp_ack'] = pkt[TCP].ack
                record['tcp_dataofs'] = pkt[TCP].dataofs
                record['tcp_reserved'] = pkt[TCP].reserved
                record['tcp_flags'] = pkt[TCP].flags
                if(pkt[TCP].flags.S):
                    record['tcp_flags_syn'] =1
                record['tcp_window'] = pkt[TCP].window
                record['tcp_chksum'] = pkt[TCP].chksum
                record['tcp_urgptr'] = pkt[TCP].urgptr
                if(pkt[TCP].flags.R):
                    record['tcp_flag_reset'] =1
                    
            df = df.append(record,ignore_index=True)
                 
            df.to_csv("test.csv")
    return df
def preprocess():
    test_frame = pd.read_csv('test.csv')
    test_frame=test_frame.drop(['Unnamed: 0'], axis=1)
    test_frame = test_frame.fillna(-1)
    with open("keysInformation.pkl", 'rb') as f:
            col_list=pickle.load(f)
            test_frame=test_frame.drop(col_list, axis=1)
   
    return test_frame.values.tolist()

datasetChange=0;
            
options = ['logistic regression ', 'Decision Tree', 'Random Forest', 'Ensemble1(LR+DT)','Ensemble2(DT+RF)','Ensemble3(RF+LR)','Ensemble4(RF+LR+DT)']
pklFiles = ['logistic.pkl','decisiontree.pkl','randomforest.pkl','decisiontreewithlogit.pkl','decisiontreewithrandom.pkl','logitwithrandomforest.pkl','logitrandomforestdecicion.pkl']
dictFileInfo=[]
Disese=['BENIGN', 'DOS', 'MIRAI', 'MITM', 'SCAN']
uploadFile=""
for i in pklFiles:
    with open(i, 'rb') as f:
        model, accuracy, precision, recall, f1 = pickle.load(f)
        dictFileInfo.append([model, int(accuracy*100), int(precision*100), int(recall*100), int(f1*100)])
def update_display_box(*args):
    global uploadFile,datasetChange
    """
    Updates the display box with the selected option from the dropdown.
    """
    if(datasetChange==1):
        test =  build_dict(uploadFile) 
        datasetChange=0;
    
    selected_option = variable.get()
    display_box.delete('1.0', tk.END)
    fileIndex=options.index(selected_option)
    model=dictFileInfo[fileIndex][0]
    accuracy=dictFileInfo[fileIndex][1]
    precision=dictFileInfo[fileIndex][2]
    recall=dictFileInfo[fileIndex][3]
    f1score=dictFileInfo[fileIndex][4]
    pre_process=preprocess()
    predicted=model.predict(pre_process)
    counts = collections.Counter(predicted)
    counts_dict = dict(counts)
    for key, value in counts_dict.items():
        display_box.insert(tk.END, f"{Disese[int(key)]}: {value}\n")
    max_key = max(counts_dict.items(), key=lambda x: x[1])[0]
    #display_box.insert(tk.END, f" In this Dataset Contain {Disese[int(max_key)]} \n")
    label1.config(text=f"Accuracy : {int(accuracy)}")
    label2.config(text=f"Precision : {int(precision)}")
    label3.config(text=f"Recall : {int(recall)}")
    label4.config(text=f"FlScore : {int(f1score)}")
    #if(int(max_key)>0):
        #messagebox.showinfo("Alert", f"Attack Detected ({Disese[int(max_key)]})!")

def upload_dataset():
    """
    Allows the user to select a file and displays the path to the selected file in the display box.
    """
    global uploadFile,datasetChange
    datasetChange=1
    file_path = filedialog.askopenfilename()
    uploadFile=file_path
    display_box.delete('1.0', tk.END)
    display_box.insert(tk.END, f"Selected file: {file_path}")
    
# Create the main window
root = tk.Tk()

# Set the window title
root.title("Network Intrusion Detector")

# Set the window size to maximum

heading = tk.Label(root, text="Network Intrusion Detection", font=("Arial", 24), bg='#800080', fg='#FFFFFF', pady=20)
heading.pack()

# Set the background color
root.configure(bg='#800080')

# Create the frame
frame = tk.Frame(root)
frame.pack()
button = tk.Button(frame, text="Upload Dataset", font=("Arial", 14), bg='#800080', fg='#FFFFFF', padx=10, pady=10,command=upload_dataset)
button.grid(row=0, column=0, sticky='W', padx=10, pady=10)

# Create the dropdown menu
variable = tk.StringVar(frame)
variable.set(options[0])
dropdown = tk.OptionMenu(frame, variable, *options)

variable.trace('w', update_display_box)

dropdown.config(width=15, font=("Arial", 14),padx=10, pady=10)
dropdown.grid(row=1, column=0, sticky='W', padx=10, pady=10)


# Create the labels
label1 = tk.Label(frame, text=f"Accuracy : {dictFileInfo[0][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
label1.grid(row=2, column=0, sticky='W', padx=10, pady=10)

label2 = tk.Label(frame, text=f"Precision : {dictFileInfo[0][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
label2.grid(row=3, column=0, sticky='W', padx=10, pady=10)

label3 = tk.Label(frame, text=f"Recall : {dictFileInfo[0][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
label3.grid(row=4, column=0, sticky='W', padx=10, pady=10)

label4 = tk.Label(frame, text=f"F1Score : {dictFileInfo[0][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
label4.grid(row=5, column=0, sticky='W', padx=10, pady=10)



# Create the display box
display_box = tk.Text(frame, width=50, height=15, font=("Arial", 14), bd=1, relief=tk.SOLID)
display_box.grid(row=0, column=1, rowspan=6, columnspan=2, padx=10, pady=10)

#button = tk.Button(root, text="Go to another page" )
#button.grid(row=6, column=0, padx=10, pady=10)

def create_second_page():
    second_page = tk.Toplevel(root)
    label = tk.Label(second_page, text="Network Attack Detector")
    second_page.title("Network Attack Detector")
    label.grid(row=0, column=0)
    
    #Logistic Regression
    heading_label = tk.Label(second_page, text="Logistic Regression", font=("Arial", 10, "bold")) 
    heading_label.grid(row=1, column=1)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[0][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=2, column=1, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[0][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=3, column=1, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[0][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=4, column=1, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[0][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=5, column=1, sticky='W', padx=10, pady=10)
    
    #Decision Tree
    heading_label = tk.Label(second_page, text="Decision Tree", font=("Arial", 10, "bold")) 
    heading_label.grid(row=1, column=2)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[1][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=2, column=2, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[1][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=3, column=2, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[1][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=4, column=2, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[1][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=5, column=2, sticky='W', padx=10, pady=10)
    
    #Random Forest
    heading_label = tk.Label(second_page, text="Random Forest", font=("Arial", 10, "bold")) 
    heading_label.grid(row=1, column=3)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[2][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=2, column=3, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[2][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=3, column=3, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[2][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=4, column=3, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[2][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=5, column=3, sticky='W', padx=10, pady=10)
    
    #LR+DT
    heading_label = tk.Label(second_page, text="LR+DT", font=("Arial", 10, "bold")) 
    heading_label.grid(row=8, column=0)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[3][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=9, column=0, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[3][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=10, column=0, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[3][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=11, column=0, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[3][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=12, column=0, sticky='W', padx=10, pady=10)
    
    #DT+RF
    heading_label = tk.Label(second_page, text="DT+RF", font=("Arial", 10, "bold")) 
    heading_label.grid(row=8, column=1)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[4][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=9, column=1, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[4][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=10, column=1, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[4][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=11, column=1, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[4][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=12, column=1, sticky='W', padx=10, pady=10)
    
    #RF+LR
    heading_label = tk.Label(second_page, text="RF+LR", font=("Arial", 10, "bold")) 
    heading_label.grid(row=8, column=2)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[5][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=9, column=2, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[5][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=10, column=2, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[5][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=11, column=2, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[5][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=12, column=2, sticky='W', padx=10, pady=10)
    
    #RF+LR+DT
    heading_label = tk.Label(second_page, text="RF+LR+DT", font=("Arial", 10, "bold")) 
    heading_label.grid(row=8, column=3)

    label1 = tk.Label(second_page, text=f"Accuracy : {dictFileInfo[6][1]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label1.grid(row=9, column=3, sticky='W', padx=10, pady=10)

    label2 = tk.Label(second_page, text=f"Precision : {dictFileInfo[6][2]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label2.grid(row=10, column=3, sticky='W', padx=10, pady=10)

    label3 = tk.Label(second_page, text=f"Recall : {dictFileInfo[6][3]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label3.grid(row=11, column=3, sticky='W', padx=10, pady=10)

    label4 = tk.Label(second_page, text=f"F1Score : {dictFileInfo[6][4]}", font=("Arial", 14), bd=0, relief=tk.SOLID, padx=10, pady=10)
    label4.grid(row=12, column=3, sticky='W', padx=10, pady=10)
    
    #space 
    label5 = tk.Label(second_page, text=" ")
    label5.grid(row=6, column= 1, sticky='W', padx=10, pady=10)
    
    label6 = tk.Label(second_page, text=" ")
    label6.grid(row=7, column= 1, sticky='W', padx=10, pady=10)
    
    
    
button = tk.Button(root, text="Result Page", command=create_second_page)
button.pack()

#def create_second_page():
    #second_page = tk.Toplevel(root)
    #label = tk.Label(second_page, text="Network Attack Detector")
    #label.grid(row=0, column=0)

#window = tk.Tk()
#button = tk.Button(root, text="Result Page", command=create_second_page)
#button.grid(row=0, column=0)
#window.mainloop()
#window.mainloop()

# Start the event loop
root.mainloop()


